In [1]:
%load_ext autoreload
%autoreload 2

from config.rnn import default
from models import RNN
import numpy as np
from functional import seq
import tensorflow as tf
from tensorpack import (TrainConfig, SyncMultiGPUTrainerParameterServer as Trainer, 
                        PredictConfig, MultiProcessDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager
from utils.validation import (Accumulator, AggregateMetric, calcu_metrics)

resnet_loc = "./data/resnet_v2_101/resnet_v2_101.ckpt"
log_dir = 'train_log/transfer/all_stages-g2-hidden/'
logger.set_logger_dir(log_dir)

[1028 08:38:10 @logger.py:94] WRN Log directory train_log/transfer/all_stages-g2-hidden/ exists! Please either backup/delete it, or use a new directory.
[1028 08:38:10 @logger.py:96] WRN If you're resuming from a previous run you can choose to keep it.
[1028 08:38:10 @logger.py:97] Select Action: k (keep) / b (backup) / d (delete) / n (new) / q (quit):
d
[1028 08:38:32 @logger.py:74] Argv: /home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1012/jupyter/kernel-cba806d0-4add-41ab-ba1a-3c75da85f415.json


In [2]:
config = default
ignore_restore = ['learning_rate', 'global_step']
save_name = "all-stages-max-micro-auc.tfmodel"

#### Split train set and test set

In [3]:
config.stages = [2, 3, 4, 5, 6]
config.proportion = {'train': 0.55, 'val': 0.0, 'test': 0.45}
config.annotation_number = None
dm = DataManager.from_config(config)
train_set = dm.get_train_set()

Group numbers:
train: 5014, validation: 0, test: 4726
Image numbers:
train: 22989, validation: 0, test: 21028


#### Gether data to train rnn

In [4]:
config.proportion = {'train': 0.8, 'val':0.2, 'test': 0.0}
config.annotation_number = 30
dm = DataManager.from_dataset(train_set, config)

Group numbers:
train: 3584, validation: 778, test: 0
Image numbers:
train: 16196, validation: 3364, test: 0


In [5]:
config.weight_decay = 0.0
config.dropout_keep_prob = 0.5
config.gamma = 2
config.use_glimpse = True
config.read_time = 5
config.batch_size = 64
config.use_hidden_dense = True

threshold = 0.5
train_data = dm.get_train_stream()
val_data = dm.get_validation_stream()
model = RNN(config, is_finetuning=False)

In [6]:
tf.reset_default_graph()
train_config = TrainConfig(model=model, dataflow=train_data,
                           callbacks=[
                               ScheduledHyperParamSetter('learning_rate', [(0, 1e-4), (15, 1e-5)]),
                               InfRunner(val_data, [AggregateMetric(config.validation_metrics, threshold)],
                                         [0, 1]),
                               ModelSaver(var_collections='model_variables'),
                               MaxSaver('micro_auc', save_name),
                           ],
                           session_init=SaverRestore(
                               model_path=resnet_loc, ignore=ignore_restore),
                           max_epoch=20, nr_tower=2)
Trainer(train_config).train()

[1028 08:38:55 @inference_runner.py:83] InferenceRunner will eval on an InputSource of size 12
[1028 08:38:56 @input_source.py:178] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[1028 08:38:56 @input_source.py:459] Setting up StagingArea for GPU prefetching ...
[1028 08:38:56 @training.py:41] Training a model of 2 towers
[1028 08:38:56 @training.py:92] Building graph for training tower 0 on device LeastLoadedDeviceSetter-/gpu:0...
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
IN

[1028 08:39:10 @base.py:212] Creating the session ...
[1028 08:39:14 @base.py:216] Initializing the session ...
[1028 08:39:14 @sessinit.py:116] Restoring checkpoint from ./data/resnet_v2_101/resnet_v2_101.ckpt ...
INFO:tensorflow:Restoring parameters from ./data/resnet_v2_101/resnet_v2_101.ckpt
[1028 08:39:15 @base.py:223] Graph Finalized.
[1028 08:39:15 @param.py:144] After epoch 0, learning_rate will change to 0.00010000
[1028 08:39:15 @concurrency.py:36] Starting EnqueueThread DataParallelInferenceRunner/QueueInput/input_queue ...
[1028 08:39:17 @concurrency.py:36] Starting EnqueueThread QueueInput/input_queue ...
[1028 08:39:17 @input_source.py:418] Pre-filling staging area ...
[1028 08:39:20 @base.py:257] Start Epoch 1 ...


100%|##########|56/56[02:02<00:00, 0.41it/s]

[1028 08:41:22 @base.py:267] Epoch 1 (global_step 56) finished, time:122.27 sec.



  0%|          |0/12[00:00<?,?it/s]/home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|##########|12/12[00:10<00:00, 1.17it/s]


...

[1028 09:19:32 @saver.py:89] Model saved to train_log/transfer/all_stages-g2-hidden/model-1064.
[1028 09:19:32 @monitor.py:361] DataParallelInferenceRunner/QueueInput/queue_size_1: 49.67
[1028 09:19:32 @monitor.py:361] QueueInput/queue_size_1: 49.414
[1028 09:19:32 @monitor.py:361] coverage: 4.099
[1028 09:19:32 @monitor.py:361] learning_rate: 1e-05
[1028 09:19:32 @monitor.py:361] loss/value: 0.031291
[1028 09:19:32 @monitor.py:361] macro_auc: 0.93818
[1028 09:19:32 @monitor.py:361] macro_f1: 0.51269
[1028 09:19:32 @monitor.py:361] mean_average_precision: 0.68781
[1028 09:19:32 @monitor.py:361] micro_auc: 0.94971
[1028 09:19:32 @monitor.py:361] micro_f1: 0.62736
[1028 09:19:32 @monitor.py:361] one_error: 0.27214
[1028 09:19:32 @monitor.py:361] ranking_loss: 0.051732
[1028 09:19:32 @monitor.py:361] ranking_mean_average_precision: 0.77136
[1028 09:19:32 @monitor.py:361] training_auc: 0.92314
[1028 09:19:32 @group.py:42] Callbacks took 10.493 sec in total. DataParallelInferenceRunner: 10.

100%|##########|56/56[01:55<00:00, 0.48it/s]

[1028 09:21:28 @base.py:267] Epoch 20 (global_step 1120) finished, time:115.58 sec.



100%|##########|12/12[00:10<00:00, 1.18it/s]


[1028 09:21:38 @saver.py:89] Model saved to train_log/transfer/all_stages-g2-hidden/model-1120.
[1028 09:21:39 @saver.py:158] Model with maximum 'micro_auc' saved.
[1028 09:21:39 @monitor.py:361] DataParallelInferenceRunner/QueueInput/queue_size_1: 49.563
[1028 09:21:39 @monitor.py:361] QueueInput/queue_size_1: 49.515
[1028 09:21:39 @monitor.py:361] coverage: 3.9727
[1028 09:21:39 @monitor.py:361] learning_rate: 1e-05
[1028 09:21:39 @monitor.py:361] loss/value: 0.031483
[1028 09:21:39 @monitor.py:361] macro_auc: 0.94005
[1028 09:21:39 @monitor.py:361] macro_f1: 0.52725
[1028 09:21:39 @monitor.py:361] mean_average_precision: 0.68972
[1028 09:21:39 @monitor.py:361] micro_auc: 0.95259
[1028 09:21:39 @monitor.py:361] micro_f1: 0.63018
[1028 09:21:39 @monitor.py:361] one_error: 0.26302
[1028 09:21:39 @monitor.py:361] ranking_loss: 0.048341
[1028 09:21:39 @monitor.py:361] ranking_mean_average_precision: 0.77406
[1028 09:21:39 @monitor.py:361] training_auc: 0.92607
[1028 09:21:39 @group.py:42

In [15]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[<tf.Variable 'EMA/QueueInput/queue_size:0' shape=() dtype=float32_ref>,
 <tf.Variable 'EMA/QueueInput/queue_size/biased:0' shape=() dtype=float32_ref>,
 <tf.Variable 'EMA/QueueInput/queue_size/local_step:0' shape=() dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/conv1/weights:0' shape=(7, 7, 3, 64) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/conv1/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_mean:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_variance:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/weights:0' shape=(1, 1, 64, 256) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bot

In [1]:
a, b = 1, 2